In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [29]:
from datetime import datetime

In [22]:
import pandas as pd

In [2]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
sessions_sql_raw = sqlContext.read.format("csv").load('session_100000.csv',header=True, inferSchema="true")

In [4]:
sessions_sql_raw.printSchema()

root
 |-- app_id: long (nullable = true)
 |-- session_id: long (nullable = true)
 |-- start_timestamp: long (nullable = true)
 |-- timezone: string (nullable = true)
 |-- timezone_offset: integer (nullable = true)
 |-- previous_sessions_duration: integer (nullable = true)
 |-- user_created_timestamp: long (nullable = true)
 |-- is_user_first_session: boolean (nullable = true)
 |-- is_session: boolean (nullable = true)
 |-- is_developer: boolean (nullable = true)
 |-- is_wau: boolean (nullable = true)
 |-- is_mau: boolean (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- locale: string (nullable = true)
 |-- os_name: string (nullable = true)
 |-- session_index: integer (nullable = true)
 |-- device_id: string (nullable = true)
 |-- user_id_hash: string (nullable = true)



In [5]:
sessions_sql = sessions_sql_raw['app_id','session_id','start_timestamp','previous_sessions_duration',
                                'user_created_timestamp','is_user_first_session','is_session','is_developer',
                                'country','city','device_id','user_id_hash']

In [6]:
sessions_sql.printSchema()

root
 |-- app_id: long (nullable = true)
 |-- session_id: long (nullable = true)
 |-- start_timestamp: long (nullable = true)
 |-- previous_sessions_duration: integer (nullable = true)
 |-- user_created_timestamp: long (nullable = true)
 |-- is_user_first_session: boolean (nullable = true)
 |-- is_session: boolean (nullable = true)
 |-- is_developer: boolean (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- user_id_hash: string (nullable = true)



In [7]:
sessions_sql = sessions_sql.filter('is_developer==false').drop('is_developer')
sessions_sql.show()

+----------------+-------------------+---------------+--------------------------+----------------------+---------------------+----------+-------+-----------+--------------------+--------------------+
|          app_id|         session_id|start_timestamp|previous_sessions_duration|user_created_timestamp|is_user_first_session|is_session|country|       city|           device_id|        user_id_hash|
+----------------+-------------------+---------------+--------------------------+----------------------+---------------------+----------+-------+-----------+--------------------+--------------------+
|4724682771660800|5558845121177764917|  1542215364580|                  25837591|         1538874289458|                false|      true|     PH|     makati|546a3d98-d540-4e7...|9943447915df3a45f...|
|4724682771660800|2201961907282901522|  1543712977293|                  35050130|         1538874289458|                false|      true|     PH|quezon city|546a3d98-d540-4e7...|9943447915df3a45f...|


In [13]:
sessions_sql.groupBy('is_user_first_session').count().show()

+---------------------+-----+
|is_user_first_session|count|
+---------------------+-----+
|                 true|13579|
|                false|86418|
+---------------------+-----+



In [14]:
sessions_psduration = sessions_sql.select('previous_sessions_duration')

Show the min and max of previous_sessions_duration

In [15]:
sessions_psduration.agg({'previous_sessions_duration': 'max'}).collect()

[Row(max(previous_sessions_duration)=1521144597)]

In [16]:
sessions_psduration.agg({'previous_sessions_duration': 'min'}).collect()

[Row(min(previous_sessions_duration)=0)]

In [17]:
sessions_psduration = sessions_psduration.toPandas()

Plot histogram is too slow

In [18]:
sessions_sql.groupBy('app_id').count().show()

+----------------+-----+
|          app_id|count|
+----------------+-----+
|4724682771660800|99997|
+----------------+-----+



In [19]:
sessions_sql.groupBy('country').count().sort('count', ascending = False).show()

+-------+-----+
|country|count|
+-------+-----+
|     US|44031|
|     IN| 7791|
|     GB| 7638|
|     CA| 3744|
|     PH| 3710|
|     ZA| 2554|
|     AU| 2383|
|     ID| 1790|
|     MY| 1099|
|     NG|  980|
|     NL|  979|
|     PT|  903|
|     RO|  866|
|     DE|  865|
|     HU|  830|
|     SA|  812|
|     CR|  796|
|     JM|  786|
|     FR|  775|
|     EG|  722|
+-------+-----+
only showing top 20 rows



In [60]:
sessions = sessions_sql.drop('app_id','session_id','is_session','device_id').toPandas()
sessions['start_timestamp'] = pd.to_datetime(sessions['start_timestamp'], unit='ms')
sessions['user_created_timestamp'] = pd.to_datetime(sessions['user_created_timestamp'], unit='ms')

In [61]:
sessions

,start_timestamp,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,city,user_id_hash
0,2018-11-14 17:09:24.580,25837591,2018-10-07 01:04:49.458,False,PH,makati,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2018-12-02 01:09:37.293,35050130,2018-10-07 01:04:49.458,False,PH,quezon city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,2018-10-10 23:52:48.666,11343848,2018-10-07 01:04:49.458,False,PH,makati,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2018-10-21 11:19:03.010,13499724,2018-10-07 01:04:49.458,False,PH,davao city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2018-11-19 23:53:45.528,32788010,2018-10-07 01:04:49.458,False,PH,makati,9943447915df3a45fd6720a026af905b6da6b56a37701b...
5,2018-10-08 11:25:13.013,5872534,2018-10-07 01:04:49.458,False,PH,davao city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
6,2018-10-21 07:17:35.109,12495345,2018-10-07 01:04:49.458,False,PH,davao city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
7,2018-10-21 12:11:35.593,14036096,2018-10-07 01:04:49.458,False,PH,davao city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
8,2018-11-04 08:33:57.287,14113619,2018-10-07 01:04:49.458,False,PH,quezon city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
9,2018-11-20 00:44:07.158,32827242,2018-10-07 01:04:49.458,False,PH,makati,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [62]:
sessions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99997 entries, 0 to 99996
Data columns (total 7 columns):
start_timestamp               99997 non-null datetime64[ns]
previous_sessions_duration    99997 non-null int32
user_created_timestamp        99997 non-null datetime64[ns]
is_user_first_session         99997 non-null bool
country                       99431 non-null object
city                          99400 non-null object
user_id_hash                  99997 non-null object
dtypes: bool(1), datetime64[ns](2), int32(1), object(3)
memory usage: 25.4 MB


In [63]:
dec1_cutoff = datetime(2018, 12, 1, 0, 0, 0)
dec14_cutoff = datetime(2018, 12, 14, 0, 0, 0)

In [64]:
dec1_cutoff

datetime.datetime(2018, 12, 1, 0, 0)

In [65]:
session_train = sessions[sessions['start_timestamp'] < dec1_cutoff]

In [66]:
session_dec1_dec14 = sessions[sessions['start_timestamp'] > dec1_cutoff]

In [67]:
session_dec1 = session_dec1_dec14[session_dec1_dec14['start_timestamp'] < dec14_cutoff]
session_dec14 = session_dec1_dec14[session_dec1_dec14['start_timestamp'] > dec14_cutoff]

In [69]:
session_dec1

,start_timestamp,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,city,user_id_hash
1,2018-12-02 01:09:37.293,35050130,2018-10-07 01:04:49.458,False,PH,quezon city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
23,2018-12-02 03:12:09.893,36803999,2018-10-07 01:04:49.458,False,PH,quezon city,9943447915df3a45fd6720a026af905b6da6b56a37701b...
213,2018-12-13 09:44:02.367,2997303,2018-11-17 23:13:23.752,True,US,lakeland highlands,207852210fbcc1e958c85f1fa0729b3acf67c1ae26c775...
214,2018-12-06 20:52:20.666,10435349,2018-11-24 00:58:27.704,False,GB,london,deaf54103e439789b069a95c4650dc6cdc24d6c9d700f1...
227,2018-12-02 12:45:13.255,31055519,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
228,2018-12-12 15:10:42.993,45301270,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
230,2018-12-03 23:50:28.523,35367496,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
231,2018-12-02 08:16:15.133,26870074,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
234,2018-12-01 22:17:35.241,25691189,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
236,2018-12-09 09:33:38.292,37320227,2018-11-28 13:21:24.170,False,TT,san fernando,8a7a5ba16ee97139d5461f78faed63c596c42f1ab3179a...
